# LangChain Prompt Templates

In [ ]:
import os
from langchain_custom_utils.helper import get_openai_api_key, get_deeplake_api_key, print_response
OPENAI_API_KEY = get_openai_api_key()
DEEPLAKE_API_KEY = get_deeplake_api_key()

LLMs operate on a straightforward principle: they accept a text input sequence and generate an output text sequence. The key factor in this process is the input text or prompt. The LangChain library has developed a comprehensive suite of objects tailored for them.

A prompt template is a preset format or blueprint to create consistent and effective prompts for large language models. It serves as a structural guide to ensure the prompt is correctly formatted. It is a guideline to properly format the input text or prompt.

In this chapter, we will apply key LangChain components such as prompt templates and output parsers, improve our previously created news summarizer with output parsers, and create a knowledge graph from text data.

The following code example illustrates how a PromptTemplate can be used with a single dynamic input for a user query.

In [ ]:
from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided, answer
with "I don't know".
Context: Quantum computing is an emerging field that leverages quantum mechanics to solve complex problems faster than classical computers.
...
Question: {query}
Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

# Create the LLMChain for the prompt
chain = LLMChain(llm=llm, prompt=prompt_template)

# Set the query you want to ask
input_data = {"query": "What is the main advantage of quantum computing over classical computing?"}

# Run the LLMChain to get the AI-generated answer
response = chain.run(input_data)

print("Question:", input_data["query"])
print("Answer:", response)

You can modify the input_data dictionary with a question of your choice.

The template functions as a formatted string featuring a `{query}` placeholder, replaced with an actual question passed to the `.run()` method. To establish a `PromptTemplate` object, two elements are essential:

1.  **input_variables**: This is a list of variable names used in the template; in this case, it comprises only the query.
2.  **template**: This is the template string, which includes formatted text and placeholders.

Once the `PromptTemplate` object is created, it can generate specific prompts by supplying the appropriate input data. This input data should be structured as a dictionary, with keys matching the variable names in the template. The crafted prompt can be forwarded to a language model to generate a response.

For more complex applications, you can construct a `FewShotPromptTemplate` with an `ExampleSelector`. This allows for selecting a subset of examples and helps effortlessly apply the few-shot learning method without the hassle of composing the entire prompt.

In [ ]:
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Create the LLMChain for the dynamic_prompt
chain = LLMChain(llm=llm, prompt=dynamic_prompt)

# Run the LLMChain with input_data
input_data = {"input": "tiger"}
response = chain.run(input_data)

print(response)

You can also save your `PromptTemplate` in your local file system in *JSON* or *YAML* format:

In [ ]:
prompt_template.save("awesome_prompt.json")

And load it back:

In [ ]:
from langchain.prompts import load_prompt
loaded_prompt = load_prompt("awesome_prompt.json")

Let’s look at some more examples using different prompt templates. We’ll see how to improve LLM responses using few-shot prompts by providing examples that direct the model to answer sarcastically using this method.

In [ ]:
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

examples = [
    {
        "query": "How do I become a better programmer?",
        "answer": "Try talking to a rubber duck; it works wonders."
    }, {
        "query": "Why is the sky blue?",
        "answer": "It's nature's way of preventing eye strain."
    }
]

example_template = """
User: {query}
AI: {answer}
"""

example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative and funny responses to users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create the LLMChain for the few_shot_prompt_template
chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)

# Run the LLMChain with input_data
input_data = {"query": "How can I learn quantum computing?"}
response = chain.run(input_data)

print(response)

The `FewShotPromptTemplate` in the example shows the effectiveness of dynamic prompts by incorporating examples from past interactions, which helps the AI better grasp the context and style of the desired response. Unlike static templates, in-context few-shot prompts offer enhanced contextual understanding, allowing the LLM to generate outputs that align more closely with the intended outcome. They also provide flexibility, enabling developers to modify and tailor prompts for specific scenarios, making it easier to experiment with different structures. As a result, dynamic prompts tend to produce higher-quality results that better meet user expectations.

Prompt templates easily integrate with other LangChain functionalities, such as chaining, and provide control over the number of examples based on the query’s length. This is beneficial for optimizing token usage and balancing the number of examples and the overall size of the prompt.

To optimize the performance of few-shot learning, providing the model with as many relevant examples as possible without exceeding the maximum context window or causing excessive processing time is crucial. The dynamic inclusion or exclusion of examples allows a balance between providing sufficient context and maintaining efficiency in the model’s operation:

In [ ]:
examples = [
    {
        "query": "How do you feel today?",
        "answer": "As an AI, I don't have feelings, but I've got jokes!"
    }, {
        "query": "What is the speed of light?",
        "answer": "Fast enough to make a round trip around Earth 7.5 times in one second!"
    }, {
        "query": "What is a quantum computer?",
        "answer": "A magical box that harnesses the power of subatomic particles to solve complex problems."
    }, {
        "query": "Who invented the telephone?",
        "answer": "Alexander Graham Bell, the original 'ringmaster'."
    }, {
        "query": "What programming language is best for AI development?",
        "answer": "Python, because it's the only snake that won't bite."
    }, {
        "query": "What is the capital of France?",
        "answer": "Paris, the city of love and baguettes."
    }, {
        "query": "What is photosynthesis?",
        "answer": "A plant's way of saying 'I'll turn this sunlight into food. You're welcome, Earth.'"
    }, {
        "query": "What is the tallest mountain on Earth?",
        "answer": "Mount Everest, Earth's most impressive bump."
    }, {
        "query": "What is the most abundant element in the universe?",
        "answer": "Hydrogen, the basic building block of cosmic smoothies."
    }, {
        "query": "What is the largest mammal on Earth?",
        "answer": "The blue whale, the original heavyweight champion of the world."
    }, {
        "query": "What is the fastest land animal?",
        "answer": "The cheetah, the ultimate sprinter of the animal kingdom."
    }, {
        "query": "What is the square root of 144?",
        "answer": "12, the number of eggs you need for a really big omelette."
    }, {
        "query": "What is the average temperature on Mars?",
        "answer": "Cold enough to make a Martian wish for a sweater and a hot cocoa."
    }
]

Instead of using the example list directly, we implement a `LengthBasedExampleSelector` like this:

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100
)

Using the `LengthBasedExampleSelector`, the code dynamically chooses and incorporates examples according to their length. This approach ensures that the final prompt remains within the specified token limit. The selector is utilized in the initialization of the `dynamic_prompt_template` object:

In [ ]:
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

So, the `dynamic_prompt_template` object employs the example_selector rather than a static set of examples. This enables the `FewShotPromptTemplate` to change the number of examples it includes **based on the length of the input query**. This approach effectively utilizes the context window, ensuring the language model has adequate context.

In [ ]:
from langchain import LLMChain, FewShotPromptTemplate, PromptTemplate
from langchain.chat_models import ChatOpenAI

from langchain.prompts.example_selector import LengthBasedExampleSelector

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Existing example and prompt definitions, and dynamic_prompt_template initialization

# Create the LLMChain for the dynamic_prompt_template
chain = LLMChain(llm=llm, prompt=dynamic_prompt_template)

# Run the LLMChain with input_data
input_data = {"query": "Who invented the telephone?"}
response = chain.run(input_data)

print(response)